In [79]:
import pandas as pd
import numpy as np
import scipy
import re
import spacy
import matplotlib.pyplot as plt 
import math
import statsmodels.api as sm
import seaborn as sns
import textwrap
import time
import random
import pickle
import requests
from tabulate import tabulate
from prettytable import PrettyTable
from bs4 import BeautifulSoup
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from gensim.utils import simple_preprocess
from gensim import corpora
from pprint import pprint
from gensim.test.utils import common_texts
from gensim.models import LdaModel
from gensim.models import CoherenceModel
import pyLDAvis.gensim_models as gensimvis
import pyLDAvis
from PyPDF2 import PdfReader
import time
import os
import tqdm
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/lucyjhampton/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [80]:
import os
os.chdir(r'/Users/lucyjhampton/Library/Mobile Documents/com~apple~CloudDocs/Jupyter/AI Bib')
# os.chdir(r'C:\Users\ljh92\iCloudDrive\Jupyter\AI Bib')

In [81]:
## LOAD NBER METADATA

nber_metadata = pd.read_csv('Data/nber_ris.csv', low_memory = False)
nber_metadata.dropna(axis = 1, inplace=True, how = 'all')
print(nber_metadata)

      Type of reference (TY)  \
0                      JOUR    
1                      JOUR    
2                      JOUR    
3                      JOUR    
4                      JOUR    
...                      ...   
33170                  JOUR    
33171                  JOUR    
33172                  JOUR    
33173                  JOUR    
33174                  JOUR    

                                           Abstract (AB)  \
0      Understanding the behavior of users online is ...   
1      We characterize how risk evolves during a cris...   
2      We examine a statewide program that identifies...   
3      We provide the first evidence on transgender e...   
4      This paper examines the impact of Generative A...   
...                                                  ...   
33170  The household production model provides a usef...   
33171  The purpose of this paper is to consider human...   
33172  In this paper, we have developed an operationa...   
33173  A topic 

In [83]:
## SEARCH FOR RELEVANT ENTRIES

condition = (
    nber_metadata['Abstract (AB)'].str.contains('artificial intelligence', case=False, na=False) |
    nber_metadata['Abstract (AB)'].str.contains('AI', case=True, na=False) | 
    nber_metadata['Abstract (AB)'].str.contains('LLM', case=True, na=False) |
    nber_metadata['Abstract (AB)'].str.contains('large language model', case=False, na=False) |
    nber_metadata['Abstract (AB)'].str.contains('Chat GPT', case=False, na=False) |
    nber_metadata['Abstract (AB)'].str.contains('generative pre-trained transformer', case=False, na=False) |
    nber_metadata['Abstract (AB)'].str.contains('neural network', case=False, na=False) |
    nber_metadata['Abstract (AB)'].str.contains('machine learning', case=False, na=False) |
    nber_metadata['Abstract (AB)'].str.contains('ML', case=True, na=False) |
    nber_metadata['Abstract (AB)'].str.contains('deep learning', case=False, na=False)
)

nber_metadata['meets_condition'] = condition

In [84]:
## STORE DATA

selected_papers = nber_metadata[nber_metadata['meets_condition'] == 1]
print(f'Number of papers meeting criteria: {len(selected_papers)}')

wp_number_list = selected_papers['URL (UR)'].to_list()

Number of papers meeting criteria: 395


In [86]:
selected_papers.to_csv('nber.csv', index = False)

In [47]:
# Configure Chrome options
chrome_options = Options()
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

# Path to your ChromeDriver (ensure this path is correct)
chrome_service = Service('/usr/local/bin/chromedriver')  # Update path if needed

# Load driver
driver = webdriver.Chrome(service=chrome_service, options=chrome_options)
nber_url = "https://www.nber.org/"
driver.get(nber_url)

# Wait for the login button to be present and click it
button = WebDriverWait(driver, 20).until(
    EC.presence_of_element_located((By.XPATH, "/html/body/div[2]/header/div/div[1]/div[3]/div/a/span"))
)
button.click()
    
# Wait for the username field to be present and enter the username
username_box = WebDriverWait(driver, 20).until(
    EC.presence_of_element_located((By.XPATH, "/html/body/div[2]/main/div[2]/div[2]/div/form/div[1]/div[1]/div/input"))
)
username_box.click()
username_box.send_keys("ljh92@cam.ac.uk")

# Wait for the password field to be present and enter the password
password_box = WebDriverWait(driver, 20).until(
    EC.presence_of_element_located((By.XPATH, "/html/body/div[2]/main/div[2]/div[2]/div/form/div[1]/div[2]/div/input"))
)
password_box.click()
password_box.send_keys("H95mgRsCD7MGEtk@")
password_box.send_keys(Keys.RETURN)

time.sleep(5)

# specify pdf download directory
download_dir = "Data/NBER_pdfs"
if not os.path.exists(download_dir):
    os.makedirs(download_dir)

# Extract cookies
cookies = driver.get_cookies()
session = requests.Session()
for cookie in cookies:
    session.cookies.set(cookie['name'], cookie['value'])

# Iterate through the list of AI papers
for link in [wp_number_list[1]]:
    pdf_url = link.strip() + ".pdf"
    
    # Download the PDF file using the session with cookies
    response = session.get(pdf_url)
    pdf_filename = os.path.join(download_dir, pdf_url.split('/')[-1])
    with open(pdf_filename, 'wb') as f:
        f.write(response.content)
    
    # Use PdfReader to extract text from the PDF file
    reader = PdfReader(pdf_filename)
    text = ""
    for page in reader.pages:
        text += page.extract_text()

    # Extract the references
    references_index = text.find("References")
    if references_index != -1:
        references_text = text[references_index:]
    else:
        print(f"'References' not found in {pdf_filename}")

    # Clean the references of headers and footers
    pattern1 = re.compile(r'\b(?:[A-Z.,:;\'"-]*[A-Z][A-Z.,:;\'"-]*\s+)+\d{1,2}\b')
    pattern2 = re.compile(r'\b\d+\.\d+\s+(?:[A-Z.,:;\'"-]+\s*)+\b')
    
    # Find all matches in the text
    matches1 = pattern1.findall(references_text)
    matches2 = pattern2.findall(references_text)
    
    # Print the matches
    for match in matches1:
        print(match)
    for match in matches2:
        print(match)
    
    # Remove the matches
    cleaned_refs = pattern1.sub('', references_text)
    cleaned_refs = pattern2.sub('', references_text)

    # Create a list of references
    print(cleaned_refs)

Extracted text from Jupyter/AI Bib/Data/NBER_pdfs/w31837.pdf following 'References':
References
Acemoglu, Daron and Todd Lensman, “Regulating Transformative Tech nologies,” July 2023.
NBER Working Paper 31461.
Aghion, Philippe, Benjamin F . Jones, and Charles I. Jones, “ Artiﬁcia l Intelligence and Economic
Growth,” in Ajay Agrawal, Joshua Gans, and Avi Goldfarb, eds., The Economics of Artiﬁcial
Intelligence: An Agenda , University of Chicago Press, 2019, pp. 237–282.
Agrawal, Ajay, Joshua Gans, and Avi Goldfarb, “Economic Policy f or Artiﬁcial Intelligence,”
Innovation Policy and the Economy , 2019, 19, 139–159.
,, and , eds, University of Chicago Press, 2019.20 CHARLES I. JONES
Aschenbrenner, Leopold, “Existential Risk and Growth,” September 2020. Glo bal Priorities
Institute Working Paper No. 6-2020.
Bostrom, Nick, “Existential Risks,” Journal of Evolution and Technology , March 2002, 9.
,Superintelligence: Paths, Dangers, Strategies , Oxford University Press, 2014.
Brynjolfsson, Er

In [83]:
# Define a regular expression pattern to match paper title and page number
pattern1 = re.compile(r'\b(?:[A-Z.,:;\'"-]*[A-Z][A-Z.,:;\'"-]*\s+)+\d{1,2}\b')
pattern2 = re.compile(r'\b\d+\.\d+\s+(?:[A-Z.,:;\'"-]+\s*)+\b')

# Find all matches in the text
matches1 = pattern1.findall(references_text)
matches2 = pattern2.findall(references_text)

# Print the matches
for match in matches1:
    print(match)
for match in matches2:
    print(match)

# Remove the matches
cleaned_refs = pattern1.sub('', references_text)
cleaned_refs = pattern2.sub('', cleaned_refs)
cleaned_refs = re.sub('\n', '', cleaned_refs)
cleaned_refs = re.sub('References', '', cleaned_refs)

# Define pattern to split the reference list
pattern = re.compile(r'(?<=\d)\.+(?=[A-Z][a-z])')

# Split the text based on the pattern
split_text = pattern.split(cleaned_refs)

# Reconstruct references, ensuring the split points are preserved
references = []
for i in range(len(split_text)):
    if i == 0:
        references.append(split_text[i].strip())
    else:
        references.append(split_text[i].strip())

for ref in references:
    print(ref)

.A.I., GROWTH, AND EXISTENTIAL RISK 21
2019.20 CHARLES I. JONES

Acemoglu, Daron and Todd Lensman, “Regulating Transformative Tech nologies,” July 2023.NBER Working Paper 31461
Aghion, Philippe, Benjamin F . Jones, and Charles I. Jones, “ Artiﬁcia l Intelligence and EconomicGrowth,” in Ajay Agrawal, Joshua Gans, and Avi Goldfarb, eds., The Economics of ArtiﬁcialIntelligence: An Agenda , University of Chicago Press, 2019, pp. 237–282
Agrawal, Ajay, Joshua Gans, and Avi Goldfarb, “Economic Policy f or Artiﬁcial Intelligence,”Innovation Policy and the Economy , 2019, 19, 139–159.,, and , eds, University of Chicago Press, Aschenbrenner, Leopold, “Existential Risk and Growth,” September 2020. Glo bal PrioritiesInstitute Working Paper No. 6-2020
Bostrom, Nick, “Existential Risks,” Journal of Evolution and Technology , March 2002, 9.,Superintelligence: Paths, Dangers, Strategies , Oxford University Press, 2014
Brynjolfsson, Erik and Andrew McAfee, The Second Machine Age: Work, Progress, and P

In [ ]:
############################################################################################################################################

                                                    # SSRN PREPRINTS

############################################################################################################################################

In [22]:
## SEARCH TERMS LIST

search_terms = ['"artificial intelligence"', 'AI', '"machine learning"', 'ML', 'GPT', '"generative pre-trained transformer"', 'ChatGPT', 'LLM', '"large language model"', '"neural network"', '"deep learning"']


In [23]:
## CONFIGURE WEBDRIVER AGAIN

# Configure Chrome options
chrome_options = Options()
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

# Path to your ChromeDriver (ensure this path is correct)
chrome_service = Service('/usr/local/bin/chromedriver')  # Update path if needed

In [24]:
## OBTAIN LIST OF PAPER URLS

# create dict of urls and search terms
url_dict = {}

## SEARCH TERMS LIST

# loop over search terms    
for search_term in search_terms:
    
    # Load driver
    driver = webdriver.Chrome(service=chrome_service, options=chrome_options)
    ssrn_url = "https://papers.ssrn.com/sol3/DisplayAbstractSearch.cfm"
    driver.get(ssrn_url)
    
    # accept cookies
    button = WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.XPATH, "/html/body/div[4]/div[2]/div/div[1]/div/div[2]/div/button[2]"))
    )
    button.click()
    
    # navigate to search bar
    searchbar = WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.XPATH, "/html/body/div[3]/div/div/div/form/div[1]/div[1]/input"))
    )
    searchbar.click()
    searchbar.send_keys(search_term)
    
    # restrict to relevant subjects
    modify_button = WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.XPATH, "/html/body/div[3]/div/div/div/form/div[4]/div/p/a"))
    )
    modify_button.click()
    
    uncheck = WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.XPATH, "/html/body/div[3]/div/div/div/form/div[5]/div[2]/div[1]/div[2]/div[1]/div[1]/div/label/span"))
    )
    uncheck.click()
    
    economics_check = WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.XPATH, "/html/body/div[3]/div/div/div/form/div[5]/div[2]/div[1]/div[2]/div[2]/div[1]/div[1]/div/label"))
    )
    economics_check.click()
    
    finance_check = WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.XPATH, "/html/body/div[3]/div/div/div/form/div[5]/div[2]/div[1]/div[2]/div[2]/div[4]/div[1]/div/label"))
    )
    finance_check.click()
    
    management_check = WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.XPATH, "/html/body/div[3]/div/div/div/form/div[5]/div[2]/div[1]/div[2]/div[2]/div[7]/div[1]/div/label"))
    )
    management_check.click()
    
    innov_check = WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.XPATH, "/html/body/div[3]/div/div/div/form/div[5]/div[2]/div[1]/div[2]/div[2]/div[9]/div[1]/div/label"))
    )
    innov_check.click()
    
    entrep_check = WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.XPATH, "/html/body/div[3]/div/div/div/form/div[5]/div[2]/div[1]/div[2]/div[2]/div[21]/div[1]/div/label"))
    )
    entrep_check.click()
    
    # search
    search_button = WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.XPATH, "/html/body/div[3]/div/div/div/form/div[1]/div[2]/button/span"))
    )
    search_button.click()
    
    # loop over pages and get urls
    paper_url_list = []
    
    # first page:
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    for a in soup.find_all('a', {'class' : 'title optClickTitle'}, href=True):
        paper_url_list.append(a['href'])

    # Scrape the following pages
    while True:
        try:
            # Locate next button
            Next = WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.CLASS_NAME, "next"))
            )
            Next.click()

            # Add random mouse movement
            x = random.randint(0, driver.get_window_size()['width'])
            y = random.randint(0, driver.get_window_size()['height'])
            driver.execute_script(f"window.scrollTo({x}, {y});")
            time.sleep(1)  # Add a wait
    
            # get the soup and the page url and append to list
            soup = BeautifulSoup(driver.page_source, 'html.parser')
            for a in soup.find_all('a', {'class' : 'title optClickTitle'}, href=True):
                paper_url_list.append(a['href'])
            
        except (NoSuchElementException, TimeoutException) as e:
            # Complete the loop
            break
    
    # append to dict
    url_dict[search_term] = paper_url_list
    print(f'Done scraping {search_term}')

# save dict of urls to storage
with open('url_dict.pickle', 'wb') as handle:
    pickle.dump(url_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

# show
url_dict

Done scraping "artificial intelligence"
Done scraping AI
Done scraping "machine learning"
Done scraping ML
Done scraping GPT
Done scraping "generative pre-trained transformer"
Done scraping ChatGPT
Done scraping LLM
Done scraping "large language model"
Done scraping "neural network"
Done scraping "deep learning"


{'"artificial intelligence"': ['https://papers.ssrn.com/sol3/papers.cfm?abstract_id=4135861',
  'https://papers.ssrn.com/sol3/papers.cfm?abstract_id=4399406',
  'https://papers.ssrn.com/sol3/papers.cfm?abstract_id=3093683',
  'https://papers.ssrn.com/sol3/papers.cfm?abstract_id=3193576',
  'https://papers.ssrn.com/sol3/papers.cfm?abstract_id=3319897',
  'https://papers.ssrn.com/sol3/papers.cfm?abstract_id=3366846',
  'https://papers.ssrn.com/sol3/papers.cfm?abstract_id=3178233',
  'https://papers.ssrn.com/sol3/papers.cfm?abstract_id=3274354',
  'https://papers.ssrn.com/sol3/papers.cfm?abstract_id=3084256',
  'https://papers.ssrn.com/sol3/papers.cfm?abstract_id=3512994',
  'https://papers.ssrn.com/sol3/papers.cfm?abstract_id=3064213',
  'https://papers.ssrn.com/sol3/papers.cfm?abstract_id=4535536',
  'https://papers.ssrn.com/sol3/papers.cfm?abstract_id=4574814',
  'https://papers.ssrn.com/sol3/papers.cfm?abstract_id=3044448',
  'https://papers.ssrn.com/sol3/papers.cfm?abstract_id=304444

In [49]:
## REMOVE DUPLICATE DICTIONARY ITEMS (preserving first item)

from collections import Counter

# Create a flattened list
flat_list = [val for sublist in url_dict.values() for val in sublist]

# Create list of unique items in flattened list
unique_urls = set(flat_list)
with open('unique_urls.pickle', 'wb') as handle:
    pickle.dump(unique_urls, handle, protocol=pickle.HIGHEST_PROTOCOL)

# Verify lengths
print(f'Length of full list: {len(flat_list)}')
print(f'Length of unique values: {len(unique_urls)}')

Length of full list: 14948
Length of unique values: 9052


In [50]:
## LOAD SSRN SCRAPER

import sys
sys.path.insert(1, 'ssrn_scraper/')
print(os.getcwd())

import importlib
import scraper
from scraper import *

# Reload the module
importlib.reload(scraper)

/Users/lucyjhampton/Library/Mobile Documents/com~apple~CloudDocs/Jupyter/AI Bib


<module 'scraper' from '/Users/lucyjhampton/Library/Mobile Documents/com~apple~CloudDocs/Jupyter/AI Bib/ssrn_scraper/scraper.py'>

In [51]:
## DEFINE DATA EXTRACTION HELPER FUNCTIONS

def get_title(soup):
    title = soup.select('meta[name = "citation_title"]')[0].get('content')
    return title

def get_authors(soup):
    authors = soup.select('meta[name = "citation_author"]')
    author_list = []
    for author in authors:
        author_list.append(author.get('content'))
    return author_list

def get_date(soup):
    date = soup.select('meta[name = "citation_publication_date"]')[0].get('content')
    return date

def get_doi(soup):
    doi = soup.select('meta[name = "citation_doi"]')[0].get('content')
    return doi

def get_keywords(soup):
    keywords = soup.select('meta[name = "citation_keywords"]')[0].get('content')
    return keywords

def get_abstract(soup):
    abstract_data = soup.find('div', {'class' : 'abstract-text'})
    abstract = abstract_data.find('p')
    return abstract.get_text().replace("\n", "")

def check_internet_connection():
    try:
        # Attempt to connect to ssrn to check for internet access
        requests.get("https://www.ssrn.com/index.cfm/en/", timeout=5)
        return True
    except requests.ConnectionError:
        return False

In [77]:
## EXTRACT PAPER INFORMATION

from bs4 import BeautifulSoup
from fake_useragent import UserAgent
from tqdm import tqdm

# Load progress from the last run (if any)
try:
    ssrn_results = pd.read_csv("ssrn_results.csv")
    processed_urls = set(ssrn_results['Link'])
    data_dict = ssrn_results.to_dict('list')
except FileNotFoundError:
    ssrn_results = pd.DataFrame(columns=['Title', 'Authors', 'Date', 'Abstract', 'Keywords', 'doi', 'Search-term', 'Source', 'Link'])
    processed_urls = set()
    data_dict = {
        'Title': [],
        'Authors': [],
        'Date': [],
        'Abstract': [],
        'Keywords': [],
        'doi': [],
        'Search-term': [],
        'Source': [],
        'Link': []
    }

# load the driver
driver = webdriver.Chrome(service=chrome_service, options=chrome_options)

# load ssrn home page to accept cookies
driver.get('https://www.ssrn.com/index.cfm/en/')
time.sleep(1)
button = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.XPATH, "/html/body/div[2]/div[2]/div/div[1]/div/div[2]/div/button[2]"))
)
button.click()

# Start run from last url
with open('unique_urls.pickle', 'rb') as handle:
    unique_urls = pickle.load(handle)
    
for url in tqdm(unique_urls, desc="Processing URLs"):
    if url in processed_urls:
        continue  # Skip URLs that have already been processed
    
    # Wait for internet connection if lost
    while not check_internet_connection():
        print("No internet connection. Pausing...")
        time.sleep(600)  # Wait for 10 minutes before checking again

    # change user agent
    ua = UserAgent()
    user_agent = ua.random
    driver.execute_cdp_cmd('Network.setUserAgentOverride', {"userAgent": user_agent})

    # load
    driver.get(url)

    # random respectful window movement
    time.sleep(3)
    x = random.randint(0, driver.get_window_size()['width'])
    y = random.randint(0, driver.get_window_size()['height'])
    driver.execute_script(f"window.scrollTo({x}, {y});")

    # get the soup
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    
    # Append to dict
    try:
        data_dict['Title'].append(get_title(soup))
        data_dict['Authors'].append(get_authors(soup))
        data_dict['Date'].append(get_date(soup))
        data_dict['Abstract'].append(get_abstract(soup))
        data_dict['Keywords'].append(get_keywords(soup))
        data_dict['doi'].append(get_doi(soup))
        data_dict['Search-term'].append('')
        data_dict['Source'].append("SSRN"),
        data_dict['Link'].append(url)
    except IndexError:
        time.sleep(1)
        page_text = soup.get_text()
        if 'This paper has been removed from SSRN at the request of the author' in page_text:
            print('paper removed')
            data_dict['Title'].append('')
            data_dict['Authors'].append('')
            data_dict['Date'].append('')
            data_dict['Abstract'].append('')
            data_dict['Keywords'].append('')
            data_dict['doi'].append('')
            data_dict['Search-term'].append('')
            data_dict['Source'].append("SSRN"),
            data_dict['Link'].append(url)
            continue
        else:
            print('paper present')
            soup = BeautifulSoup(driver.page_source, 'html.parser')
            data_dict['Title'].append(get_title(soup))
            data_dict['Authors'].append(get_authors(soup))
            data_dict['Date'].append(get_date(soup))
            data_dict['Abstract'].append(get_abstract(soup))
            data_dict['Keywords'].append(get_keywords(soup))
            data_dict['doi'].append(get_doi(soup))
            data_dict['Search-term'].append('')
            data_dict['Source'].append("SSRN"),
            data_dict['Link'].append(url)

    # create df out of dict
    ssrn_results = pd.DataFrame(data_dict)
    
    # save progress, add url to processed url list
    if len(ssrn_results) % 10 == 0:
        try:
            if len(ssrn_results) - len(pd.read_csv("ssrn_results.csv")) == 10:
                ssrn_results.to_csv("ssrn_results.csv", index=False)
            else:
                print('warning!')
                break
        except FileNotFoundError:
            ssrn_results.to_csv("ssrn_results.csv", index=False)
    processed_urls.add(url)

# Display the DataFrame
ssrn_results = pd.DataFrame(data_dict)
ssrn_results

Processing URLs: 100%|████████████████████| 9052/9052 [1:05:30<00:00,  2.30it/s]


,Title,Authors,Date,Abstract,Keywords,doi,Search-term,Source,Link
0,Semicovariances and Machine Learning Methods i...,"['Kim, Taeyoon', 'Kim, Byung-June', 'Jang, Bon...",2021/02/18,We adopt the semicovariance decomposition meth...,"Realized Volatility, Forecasting, Machine Lear...",10.2139/ssrn.3787938,NaN,SSRN,https://papers.ssrn.com/sol3/papers.cfm?abstra...
1,"Demand Prediction, Predictive Shipping, and Pr...","['Li, Xiaocheng', 'Zheng, Yufeng', 'Zhou, Zhen...",2019/03/12,"In this paper, we study the problems of multip...","Demand Prediction, Predictive Shipping, Produc...",NaN,NaN,SSRN,https://papers.ssrn.com/sol3/papers.cfm?abstra...
2,Accuracy of Explanations of Machine Learning M...,"['Alonso, Andrés', 'Carbó, José Manuel']",2022/06/23,One of the biggest challenges for the applicat...,"synthetic datasets, artificial intelligence, i...",10.2139/ssrn.4144780,NaN,SSRN,https://papers.ssrn.com/sol3/papers.cfm?abstra...
3,Advances in Behavioral Economics and Finance L...,"['Puaschunder, Julia M.']",2021/12/05,Advances in Behavioral Economics and Finance L...,"Artificial Intelligence, AI, Behavioral Econom...",10.2139/ssrn.4226227,NaN,SSRN,https://papers.ssrn.com/sol3/papers.cfm?abstra...
4,A Framework for Online Investment Decisions,"['Paskaramoorthy, Andrew', 'Gebbie, Tim', 'van...",2020/04/02,The artificial segmentation of the investment ...,"online adaptive learning, portfolio management...",NaN,NaN,SSRN,https://papers.ssrn.com/sol3/papers.cfm?abstra...
...,...,...,...,...,...,...,...,...,...
9047,Managing Supply and Demand for the Performing ...,"[Nattamai Kannan, Karthik Babu]",2021/11/19,The COVID-19 pandemic forced performing arts o...,"demand estimation, econometrics, super learner...",10.2139/ssrn.3970201,,SSRN,https://papers.ssrn.com/sol3/papers.cfm?abstra...
9048,A Damped Diffusion Framework for Financial Mod...,"[Li, Minqiang]",2009/08/12,Asset price bubbles can arise unintentionall...,"Damped diffusion, asset price bubbles, marting...",10.2139/ssrn.1482389,,SSRN,https://papers.ssrn.com/sol3/papers.cfm?abstra...
9049,Artificial Neural Network Models for Forecasti...,"[Al-Fattah, Saud]",2013/02/12,Energy market volatility affects macroeconomic...,"Oil Prices, Volatility, Modeling Oil Market, A...",10.2139/ssrn.2216337,,SSRN,https://papers.ssrn.com/sol3/papers.cfm?abstra...
9050,Application of evolutionary computation techni...,"[Chattopadhyay, Subhajit]",2024/01/21,The current study reviews various literatures ...,"Evolutionary algorithms, evolutionary computat...",10.2139/ssrn.4701751,,SSRN,https://papers.ssrn.com/sol3/papers.cfm?abstra...


In [78]:
ssrn_results.to_csv('ssrn_results.csv', index=False)